# check oof df

In [1]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop import get_valid_values_dict, concat_valid_input_info, get_oof_df, get_key_df
from dss_model import get_model
from dss_dataloader import get_loader


In [2]:
output_dir = "/kaggle/working"
# exp_name = "exp022_dense_skffolds_epoch10"
# exp_name = "exp023_alldata_skf_dense_epoch30"
exp_name = "exp025_alldata_skf_lstmenchead_epoch30"
folds = [0, 1, 2, 3, 4]
# folds = [0]
oof_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    # df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"raw_oof_df_fold{i}.parquet"))
    df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"fold{i}_best_oof_df.parquet"))
    oof_df = pd.concat([oof_df, df], axis=0)


print(len(oof_df["series_id"].unique()))   


fold 0
fold 1
fold 2
fold 3
fold 4
277


In [3]:
oof_df = oof_df.sort_values(["series_id", "step"])
display(oof_df)

oof_df = oof_df[["series_id", "fold","step", "second", "event_pred", "class_pred"]]


,series_id,step,anglez,enmo,second,date,anglez_mean_36,anglez_std_36,anglez_mean_60,anglez_std_60,event,event_onset,event_wakeup,series_date_key,series_date_key_str,fold,event_pred,class_pred,class_target
0,038441c925bb,0.0,2.636700,0.0217,0,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2,0.223451,0.237817,0.0
1,038441c925bb,1.0,2.636800,0.0215,5,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.177456,0.0
2,038441c925bb,2.0,2.637000,0.0216,10,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.142563,0.0
3,038441c925bb,3.0,2.636800,0.0213,15,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.125547,0.0
4,038441c925bb,4.0,2.636800,0.0215,20,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.117468,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25500432,fe90110788d2,592375.0,-27.277500,0.0204,35,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.000770,-1.0
25500433,fe90110788d2,592376.0,-27.032499,0.0233,40,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.000771,-1.0
25500434,fe90110788d2,592377.0,-26.841200,0.0202,45,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.000771,-1.0
25500435,fe90110788d2,592378.0,-26.723900,0.0199,50,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.000771,-1.0


In [4]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn

def detect_event_from_classpred(df, N=300, maxpool_kernel_size=41):
    df = df.copy()

    # series_idでgroupbyして、class_predに対して対象の列のデータから
    # 前のN個の列までのデータの平均をとる
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x.rolling(N, min_periods=1).mean()
    )
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1]
    )
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - 1) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(
        maxpool_kernel_size, stride=1, padding=maxpool_padding
    )
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    df["onset"] = np.clip(-df["event_pred"], 0, 1)
    df["wakeup"] = np.clip(df["event_pred"], 0, 1)
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)

    return df

def make_submission_df(df, threshold=0.1):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [5]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["step"].notnull()].copy()
# train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())


,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [7]:
%%time
max_kernel_size = 3
tmp_score = 0
for average_size_ in range(108, 301, 12):
    print("average_size", average_size_, "max_kernel_size", max_kernel_size)
    fold_score_list = []
    for fold in range(5):
        fold_df = oof_df[oof_df["fold"] == fold].copy()
        df = detect_event_from_classpred(fold_df, N=average_size_, maxpool_kernel_size=max_kernel_size)
        sub_df = make_submission_df(df, threshold=0.1)
        if len(sub_df) == 0:
            print("event not detected")
            continue
        scoring_event = train_event_df[train_event_df["series_id"].isin(sub_df["series_id"].unique())].copy()
        score_ = score(scoring_event, sub_df)
        print("\t", fold, score_)
        fold_score_list.append(score_)
    
    fold_mean_score = np.mean(fold_score_list)
    print("fold_mean_score", fold_mean_score)
    if tmp_score > fold_mean_score:
        break
    tmp_score = fold_mean_score
    average_size = average_size_


average_size 60 max_kernel_size 3
	 0 0.713613178111133
	 1 0.7377915548436966
	 2 0.7265822696880437
	 3 0.7319002742809615
	 4 0.7580908187029392
fold_mean_score 0.7335956191253548
average_size 72 max_kernel_size 3
	 0 0.7153237784389381
	 1 0.7438069253575605
	 2 0.729462986753828
	 3 0.7359400812559742
	 4 0.7602191691972788
fold_mean_score 0.7369505882007159
average_size 84 max_kernel_size 3
	 0 0.7183636586022286
	 1 0.7449764195818935
	 2 0.7316768254079933
	 3 0.7378877343020356
	 4 0.7625088390224475
fold_mean_score 0.7390826953833198
average_size 96 max_kernel_size 3
	 0 0.7187661961022414
	 1 0.7460728045384504
	 2 0.7333224091579857
	 3 0.7397601516964973
	 4 0.763717752302829
fold_mean_score 0.7403278627596008
average_size 108 max_kernel_size 3
	 0 0.7198817886050627
	 1 0.7470336668089357
	 2 0.7327752749645219
	 3 0.7407804312411063
	 4 0.7642118667570061
fold_mean_score 0.7409366056753266
average_size 120 max_kernel_size 3
	 0 0.7214501876061433
	 1 0.7459380756401104
	

In [17]:
tmp_score = 0
max_kernel_size_ = max_kernel_size
for max_kernel_size in range(23, 47, 2):
    print("average_size", average_size, "max_kernel_size", max_kernel_size)
    fold_score_list = []
    for fold in range(5):
        fold_df = oof_df[oof_df["fold"] == fold].copy()
        df = detect_event_from_classpred(fold_df, N=average_size, maxpool_kernel_size=max_kernel_size)
        sub_df = make_submission_df(df, threshold=0.1)
        if len(sub_df) == 0:
            print("event not detected")
            continue
        scoring_event = train_event_df[train_event_df["series_id"].isin(sub_df["series_id"].unique())].copy()
        score_ = score(scoring_event, sub_df)
        print("\t", fold, score_)
        fold_score_list.append(score_)
    fold_mean_score = np.mean(fold_score_list)
    print("fold_mean_score", fold_mean_score)
    if tmp_score > fold_mean_score:
        break
    tmp_score = fold_mean_score
    max_kernel_size_ = max_kernel_size


average_size 132 max_kernel_size 23
	 0 0.724403774474895
	 1 0.7466996925706759
	 2 0.733359196244419
	 3 0.7428407100814112
	 4 0.7665582062866216
fold_mean_score 0.7427723159316045
average_size 132 max_kernel_size 25
	 0 0.7244147893915133
	 1 0.7468596036077161
	 2 0.7333961601841708
	 3 0.7430038284062317
	 4 0.7679725929659207
fold_mean_score 0.7431293949111105
average_size 132 max_kernel_size 27
	 0 0.724549307245415
	 1 0.7468060000588383
	 2 0.7333802912728162
	 3 0.7430932171371827
	 4 0.7680669660013548
fold_mean_score 0.7431791563431214
average_size 132 max_kernel_size 29
	 0 0.7246100830543454
	 1 0.7468204406409508
	 2 0.7334070390980225
	 3 0.7429243837785517
	 4 0.7682069949298671
fold_mean_score 0.7431937883003475
average_size 132 max_kernel_size 31
	 0 0.7245009949296393
	 1 0.7468602992582527
	 2 0.7334222748204757
	 3 0.7429913452417163
	 4 0.7682372541061349
fold_mean_score 0.7432024336712437
average_size 132 max_kernel_size 33
	 0 0.7245646883777814
	 1 0.74681056

KeyboardInterrupt: 

In [ ]:
fold_score_list = []
for fold in range(5):
    fold_df = oof_df[oof_df["fold"] == fold].copy()
    df = detect_event_from_classpred(fold_df, N=average_size, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.01)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    scoring_event = train_event_df[train_event_df["series_id"].isin(sub_df["series_id"].unique())].copy()
    score_ = score(scoring_event, sub_df)
    print("\t", fold, score_)
    fold_score_list.append(score_)
fold_mean_score = np.mean(fold_score_list)

print(average_size, max_kernel_size_)
print(fold_mean_score)


	 0 0.733906533507694
	 1 0.7604374307165193
	 2 0.741196874883638
	 3 0.7512953751506489
	 4 0.77565735233906
144 31
0.752498713319512


In [ ]:
df = df[["series_id", "step", "class_pred", "onset", "wakeup"]].copy()
df = df.rename(columns={"class_pred": "sleep"})


In [ ]:
df


,series_id,step,sleep,onset,wakeup
0,05e1944c3818,0.0,0.101053,0.0,0.081247
1,05e1944c3818,1.0,0.080924,-0.0,0.000000
2,05e1944c3818,2.0,0.070058,-0.0,0.000000
3,05e1944c3818,3.0,0.065724,-0.0,0.000000
4,05e1944c3818,4.0,0.063228,-0.0,0.000000
...,...,...,...,...,...
24792248,f0482490923c,761935.0,0.005161,0.0,-0.000000
24792249,f0482490923c,761936.0,0.005129,0.0,-0.000000
24792250,f0482490923c,761937.0,0.005094,0.0,-0.000000
24792251,f0482490923c,761938.0,0.005050,0.0,-0.000000


In [ ]:
df = detect_event_from_classpred(oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size)
sub_df = make_submission_df(df, threshold=0.01)
overall_score = score(train_event_df, sub_df)
print(overall_score)


0.7492792918689151


In [ ]:



os.makedirs(os.path.join(output_dir, "_overall_oof", exp_name), exist_ok=True)
df.to_parquet(os.path.join(output_dir, "_overall_oof", exp_name, "oof_df_foldmean.parquet"))


KeyboardInterrupt: 